In [2]:
### Credits
# - https://towardsdatascience.com/an-introduction-to-neural-networks-with-implementation-from-scratch-using-python-da4b6a45c05b

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
def relu(z):
    a = np.maximum(0,z)
    return a

def initialize_params(layer_sizes):
    params = {}
    for i in range(1, len(layer_sizes)):
        params['W' + str(i)] = np.random.randn(layer_sizes[i], layer_sizes[i-1])*0.01
        params['B' + str(i)] = np.random.randn(layer_sizes[i],1)*0.01
    return params

In [5]:
def forward_propagation(X_train, params):
    layers = len(params)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], X_train) + params['B' + str(i)]
            values['A' + str(i)] = relu(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], values['A' + str(i-1)]) + params['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = relu(values['Z' + str(i)])
    return values

In [6]:
def compute_cost(values, Y_train):
    layers = len(values)//2
    Y_pred = values['A' + str(layers)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

In [7]:
def backward_propagation(params, values, X_train, Y_train):
    layers = len(params)//2
    m = len(Y_train)
    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            dA = 1/m * (values['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(params['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(values['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,values['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads


In [8]:
def update_params(params, grads, learning_rate):
    layers = len(params)//2
    params_updated = {}
    for i in range(1,layers+1):
        params_updated['W' + str(i)] = params['W' + str(i)] - learning_rate * grads['W' + str(i)]
        params_updated['B' + str(i)] = params['B' + str(i)] - learning_rate * grads['B' + str(i)]
    return params_updated

In [9]:
def model(X_train, Y_train, layer_sizes, num_iters, learning_rate):
    params = initialize_params(layer_sizes)
    for i in range(num_iters):
        values = forward_propagation(X_train.T, params)
        cost = compute_cost(values, Y_train.T)
        grads = backward_propagation(params, values,X_train.T, Y_train.T)
        params = update_params(params, grads, learning_rate)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return params

In [10]:
def compute_accuracy(X_train, X_test, Y_train, Y_test, params):
    values_train = forward_propagation(X_train.T, params)
    values_test = forward_propagation(X_test.T, params)
    train_acc = np.sqrt(mean_squared_error(Y_train, values_train['A' + str(len(layer_sizes)-1)].T))
    test_acc = np.sqrt(mean_squared_error(Y_test, values_test['A' + str(len(layer_sizes)-1)].T))
    return train_acc, test_acc

def predict(X, params):
    values = forward_propagation(X.T, params)
    predictions = values['A' + str(len(values)//2)].T
    return predictions

In [11]:
# input
input = np.array([[-0.9], [-0.8], [-0.7],[-0.6], [-0.5], [-0.4], [-0.3], [-0.2],[-0.1],
                   [ 0.0], [.1], [0.2], [0.3], [0.4], [0.5], [0.6], [0.7], [0.8], [0.9], [1.0]])

# training output
target = np.array([[-0.9], [-0.8], [-0.7],[-0.6], [-0.5], [-0.4], [-0.3], [-0.2],[-0.1],
                   [ 0.0], [.1], [0.2], [0.3], [0.4], [0.5], [0.6], [0.7], [0.8], [0.9], [1.0]])


In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(input, target, test_size = 0.2) 

In [13]:
# Small network
layer_sizes = [1, 2, 1]

# Large network
#layer_sizes = [1, 100, 100, 1]

In [14]:
num_iters = 1000                                                                  #set number of iterations over the training set(also known as epochs in batch gradient descent context)
learning_rate = 0.01                                                              #set learning rate for gradient descent
params = model(X_train, Y_train, layer_sizes, num_iters, learning_rate)           #train the model
train_acc, test_acc = compute_accuracy(X_train, X_test, Y_train, Y_test, params)  #get training and test accuracy
print('Root Mean Squared Error on Training Data = ' + str(train_acc))
print('Root Mean Squared Error on Test Data = ' + str(test_acc))

Cost at iteration 1 = 2.5120981670829066

Cost at iteration 2 = 2.493391259144308

Cost at iteration 3 = 2.4801709385941324

Cost at iteration 4 = 2.4708212668320897

Cost at iteration 5 = 2.464202042581583

Cost at iteration 6 = 2.4595087061505074

Cost at iteration 7 = 2.4561735003176173

Cost at iteration 8 = 2.453795481183543

Cost at iteration 9 = 2.452090340577406

Cost at iteration 10 = 2.450859990746385

Cost at iteration 11 = 2.4499634407448156

Cost at iteration 12 = 2.4493010801772903

Cost at iteration 13 = 2.4488024875191368

Cost at iteration 14 = 2.448417827682616

Cost at iteration 15 = 2.4481117881026773

Cost at iteration 16 = 2.447859281986046

Cost at iteration 17 = 2.447641725425007

Cost at iteration 18 = 2.4474457844959714

Cost at iteration 19 = 2.4472633246214563

Cost at iteration 20 = 2.4470876091327076

Cost at iteration 21 = 2.4469136541228678

Cost at iteration 22 = 2.446737697311505

Cost at iteration 23 = 2.446556822123949

Cost at iteration 24 = 2.44636

In [ ]:
# Try with the larger layer sizes to see improvement